# Model Training
This notebook describes the construction, training and testing of our model.

### Loading the [dataset](https://datashare.is.ed.ac.uk/handle/10283/2791)
The files are enumerated and organized into noisy, clean pairs. The audio is loaded, resampled from 48 kHz to 16 kHz and regained such that the loudest sample has absolute value 1. Only a small random selection of the training dataset is used.

In [ ]:
import os
import librosa
import numpy as np
import itertools
import random
from IPython.display import clear_output

def peak_amplitude(samples):
    return max(abs(np.min(samples)), abs(np.max(samples)))

TARGET_SR = 16000
def load_audio(path, target_sr=TARGET_SR):
    if target_sr == 48000:
        y, sr = librosa.load(path)
    else:
        y, sr = librosa.load(path, sr=target_sr, res_type='kaiser_fast')
    return y / peak_amplitude(y)

def progressbar(it, prefix="", size=100):
    count = len(it)
    def show(j):
        x = int(size*j/count)
        clear_output(wait=True)
        print("%s[%s%s] %i/%i\r" % (prefix, "#"*x, "."*(size-x), j, count)) 
    show(0)
    for j, item in enumerate(it):
        yield item
        show(j + 1)
        
def load_file_pairs(pairs):
    result = []
    for pair in progressbar(pairs): # A progressbar is shown for the lengthy loading process.
        result.append(tuple(map(load_audio, pair)))
    return result

In [ ]:
noisy_trainset_28spk_directory = "DS_10283_2791/noisy_trainset_28spk_wav"
clean_trainset_28spk_directory = "DS_10283_2791/clean_trainset_28spk_wav"
noisy_trainset_56spk_directory = "DS_10283_2791/noisy_trainset_56spk_wav"
clean_trainset_56spk_directory = "DS_10283_2791/clean_trainset_56spk_wav"
trainset_files = [(os.path.join(dirs[0], filename), os.path.join(dirs[1], filename))
                  for dirs in [(noisy_trainset_28spk_directory, clean_trainset_28spk_directory),
                                 (noisy_trainset_56spk_directory, clean_trainset_56spk_directory)]
                  for filename in os.listdir(dirs[0])]

TRAINSET_SIZE = 2000
random.shuffle(trainset_files)
trainset_files = trainset_files[:TRAINSET_SIZE]
trainset_audio = load_file_pairs(trainset_files)

In [ ]:
noisy_testset_directory = "DS_10283_2791/noisy_testset_wav"
clean_testset_directory = "DS_10283_2791/clean_testset_wav"
test_files = [(os.path.join(noisy_testset_directory, filename),
                  os.path.join(clean_testset_directory, filename))
                 for filename in os.listdir(noisy_testset_directory)]

test_audio = load_file_pairs(test_files)

### Construction of the model
We begin with a structure similar to a WaveNet. The main difference being that the padding of our dilated convolutional layers is non-causal. The sum of the skip-connections is then entered into a series of convolutional layers.  
The input of the model will be a short slice of noisy audio (a subsequence of samples) and the output is meant to be a slice of the same length with denoised audio.

In [ ]:
from keras.models import Model
from keras.layers import Conv1D, Flatten, Dense, \
  Input, Activation, Add, Multiply

# This code served as inspiration:
# https://github.com/usernaamee/keras-wavenet
def wavenet_residual_block(filters, kernel_size, dilation_rate):
    def f(input_):
        conv = Conv1D(filters, kernel_size, 
                      dilation_rate=dilation_rate,
                      padding='same')(input_)
        tanh = Activation('tanh')(conv)
        sigmoid = Activation('sigmoid')(conv)
        merged = Multiply()([tanh, sigmoid])
        out = Conv1D(1, 1, padding='same')(merged)
        residual_out = Add()([out, input_])
        skip_out = Activation('relu')(out)
        return residual_out, skip_out
    return f

def wavenet_convolutional_layers(filters, kernel_size, depth):
    def f(input_):
        residual_out = input_
        skip_connections = []
        for i in range(1, depth+1):
            dilation_rate = 2**(i % 9)
            residual_out, skip_out = wavenet_residual_block(
                filters, kernel_size, dilation_rate)(residual_out)
            skip_connections.append(skip_out)
        sum_ = Add()(skip_connections)
        act = Activation('relu')(sum_)
        return act
    return f

def wavenet(input_size):
    input_ = Input(shape=(input_size, 1))
    net = wavenet_convolutional_layers(128, 3, 30)(input_)
    net = Conv1D(2048, 3, padding='same')(net)
    net = Conv1D(256, 3, padding='same')(net)
    net = Conv1D(1, 1, padding='same')(net)
    model = Model(input_, net)
    model.compile(loss='MAE', optimizer='adam')
    return model

SLICE_SIZE = 2048
model = wavenet(SLICE_SIZE)

### Training
The training samples consist of pairs of noisy and clean audio slices. A generator is used to create these slices and organize them into mini-batches for the Keras API to consume. The placement of the slices within the audio files are random, as to avoid any bias caused by repeated slices. Setting a high slice density parameter allows more overlapping slices to be generated.

In [ ]:
def batch_count(noisy_clean_audio_pairs, slice_size, batch_size, slice_density=1.0):
    slices = sum(int(clean.shape[0] / slice_size * slice_density)
                 for noisy, clean in noisy_clean_audio_pairs)
    return slices // batch_size

def slice_generator(noisy_clean_audio_pairs, slice_size, batch_size, slice_density=1.0):
    # Memory optimalization: Instead of allocating a new numpy array for every batch,
    # two alternating buffers are used. Having one buffer does not suffice, as 
    # the Keras API allows no modifications to arrays passed to it.
    batch = (np.zeros((batch_size, slice_size)), np.zeros((batch_size, slice_size)))
    next_batch = (np.zeros((batch_size, slice_size)), np.zeros((batch_size, slice_size)))
    batch_x, batch_y = batch
    while True:
        i = 0
        for noisy_audio, clean_audio in noisy_clean_audio_pairs:
            slice_count = int(clean_audio.shape[0] / slice_size * slice_density)
            for _ in range(slice_count):
                j = np.random.randint(clean_audio.shape[0] - slice_size - 1)
                batch_x[i, :] = noisy_audio[j:j + slice_size]
                batch_y[i, :] = clean_audio[j:j + slice_size]
                i += 1
                if i == batch_size:
                    yield batch
                    batch, next_batch = next_batch, batch
                    batch_x, batch_y = batch
                    i = 0

In [ ]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import train_test_split

checkpoint = ModelCheckpoint(
    'models/model.hdf5',
    save_freq='epoch',
    save_best_only=True)

rlrp = ReduceLROnPlateau(
    min_delta=0.0005,
    patience=3, 
    verbose=1,
    monitor='val_loss')

es = EarlyStopping(
    patience=4, 
    verbose=1,
    monitor='val_loss',
    restore_best_weights=True)

BATCH_SIZE = 10
SLICE_DENSITY = 1.0
EPOCHS = 20
VALIDATION_SPLIT = 0.4

random.shuffle(trainset_audio)
train_audio, val_audio = train_test_split(trainset_audio, test_size=VALIDATION_SPLIT)

train_gen = slice_generator(train_audio, SLICE_SIZE, BATCH_SIZE, SLICE_DENSITY)
train_batches = batch_count(train_audio, SLICE_SIZE, BATCH_SIZE, SLICE_DENSITY)

val_gen = slice_generator(val_audio, SLICE_SIZE, BATCH_SIZE, SLICE_DENSITY)
val_batches = batch_count(val_audio, SLICE_SIZE, BATCH_SIZE, SLICE_DENSITY)

model.fit(x=train_gen,
          steps_per_epoch=train_batches,
          validation_data=val_gen,    
          validation_steps=val_batches,
          epochs=EPOCHS,
          callbacks=[checkpoint, rlrp, es])

### Testing
The loss is calculated for the entire testing dataset. Individual test files are evaluated manually.

In [ ]:
TEST_SLICE_DENSITY = 2.0
test_gen = slice_generator(test_audio, SLICE_SIZE, BATCH_SIZE, TEST_SLICE_DENSITY)
test_batches = batch_count(test_audio, SLICE_SIZE, BATCH_SIZE, TEST_SLICE_DENSITY)
model.evaluate(x=test_gen, steps=test_batches)

In [ ]:
import IPython.display as ipd
import math 

def _model_predict(slice):
    return np.array(model.predict(slice.reshape((1, SLICE_SIZE)))[0]).reshape((SLICE_SIZE))

def _denoise_audio(audio, result):
    slice_count = audio.shape[0] // SLICE_SIZE
    for i in range(0, slice_count * SLICE_SIZE, SLICE_SIZE):
        result[i:i+SLICE_SIZE] += _model_predict(audio[i:i+SLICE_SIZE])
        
def denoise_audio(audio):
    # The input is rescaled in the same way as was the training data.
    peak = peak_amplitude(audio)
    npad = math.ceil(audio.shape[0] / SLICE_SIZE) * SLICE_SIZE
    padded = np.pad(audio, (0, npad), 'reflect')
    result = np.zeros(padded.shape)
    _denoise_audio(padded / peak, result)
    return result[:audio.shape[0]] * peak

# Run the denoising for multiple rounds with varying time offsets, then average the results.
# This is meant to eliminate random noise introduced by the model itself.
def denoise_audio_multiple(audio, n=8):
    peak = peak_amplitude(audio)
    result = np.zeros((audio.shape[0] + SLICE_SIZE))
    for offset in range(0, SLICE_SIZE - 1, SLICE_SIZE // n):
        _denoise_audio(audio[offset:] / peak, result[offset:])
    clipped = result[:audio.shape[0]]
    average = clipped / math.ceil(SLICE_SIZE / (SLICE_SIZE // n))
    return average  * peak

def do_test(noisy_clean_path_pair):
    tsr = TARGET_SR
    noisy_res, _ = librosa.load(noisy_clean_path_pair[0], sr=tsr, res_type='kaiser_best')
    result = denoise_audio(noisy_res)
    noisy, sr = librosa.load(noisy_clean_path_pair[0])
    clean, sr = librosa.load(noisy_clean_path_pair[1])
    clean_res = librosa.resample(clean, sr, tsr, res_type='kaiser_best')[:noisy_res.shape[0]]
    ipd.display(ipd.Audio(noisy_res, rate=tsr))
    ipd.display(ipd.Audio(result, rate=tsr))
    ipd.display(ipd.Audio(noisy_res - result, rate=tsr))
    ipd.display(ipd.Audio(noisy_res - clean_res, rate=tsr))
    
do_test(test_files[1])